<a href="https://colab.research.google.com/github/pawel0508/MachineLearning_UcznieNienadzorowane/blob/main/pca_03_wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

np.set_printoptions(precision=4, suppress=True, edgeitems=5, linewidth=200)

In [2]:
df_raw = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)
df = df_raw.copy()
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [4]:
data = df.iloc[:,1:]
target = df.iloc[:, 0]

In [5]:
target.value_counts()

2    71
1    59
3    48
Name: 0, dtype: int64

In [15]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, target)

print(f'x train shape: {x_train.shape}')
print(f'x test shape: {x_test.shape}')
print(f'y train shape: {y_train.shape}')
print(f'y test shape: {y_test.shape}')

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_std = scaler.fit_transform(x_train)
x_test_std = scaler.transform(x_test)

print(x_train_std[:10])

x train shape: (133, 13)
x test shape: (45, 13)
y train shape: (133,)
y test shape: (45,)
[[-0.6428 -0.973  -0.3965 -0.5414 -1.0284 -0.4499 -1.4511  1.9064 -0.6469  0.1395 -0.9048 -1.5342 -0.3563]
 [ 0.4701  1.4912  0.403   1.1129  0.1877 -0.7674 -1.2829  0.5509 -0.3655  0.9482 -1.1184 -1.4637 -0.0438]
 [-0.4802 -0.8623 -1.196  -0.7519  0.0446 -0.4181 -0.6298  1.3483 -1.7546  0.2707  0.0775 -1.4214 -0.9814]
 [ 1.0578 -0.3639  1.5154  0.0301  0.5454  1.0586  0.9137  0.0725  0.2499 -0.2758  1.2307  1.1301  0.4719]
 [-0.7553  0.236  -0.327   0.8121 -0.6707 -0.958  -0.2044  2.3848 -0.348  -1.0626 -0.435   0.9891 -1.4033]
 [-1.5431 -0.1516  1.4459  2.7671 -0.5277 -0.2434  0.1914  1.7469  0.2499 -0.9314  0.0348 -0.2232 -0.9314]
 [-0.1926  2.1372  0.403   0.6617 -0.9569 -0.9262 -1.3818  0.8699 -1.3326  1.1012 -1.8018 -1.0408 -0.4344]
 [ 0.3325 -0.2993  1.0982 -0.7519  0.1877  1.138   1.171  -0.4059  0.074   0.3799  0.4619  0.3406  1.5814]
 [-0.9679  1.4635 -0.5703 -0.2406  0.9031 -1.4343 -1.2

In [18]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 3)
x_train_pca = pca.fit_transform(x_train_std)
x_test_pca = pca.transform(x_test_std)
x_train_pca.shape


(133, 3)

Wyjaśniona wariancja

In [23]:
result = pd.DataFrame(data = {'explained_variance_ratio' : pca.explained_variance_ratio_})
result['cum_ratio'] = np.cumsum(result['explained_variance_ratio'])

In [24]:
result['component'] = result.index + 1

In [25]:
result 

,explained_variance_ratio,cum_ratio,component
0,0.364285,0.364285,1
1,0.186960,0.551244,2
2,0.117339,0.668584,3


In [28]:
fig = go.Figure(data = [go.Bar(x = result['component'], y = result['explained_variance_ratio'], name = 'explained_variance_ratio'),
          go.Scatter(x = result['component'], y = result['cum_ratio'], name = 'cum ratio')],
          layout = go.Layout(title=f'PCA - {pca.n_components_} components', width=950, template='simple_white'))
fig.show()

In [29]:
x_train_df_pca = pd.DataFrame(data = np.c_[x_train_pca, y_train], columns = ['pca_1', 'pca_2', 'pca_3', 'target'])
x_train_df_pca.head()

,pca_1,pca_2,pca_3,target
0,2.328158,0.530719,-1.158080,3.0
1,2.673677,-1.660807,0.334011,3.0
2,1.831974,0.748373,-1.597253,2.0
3,-2.107586,-0.543565,1.413820,1.0
4,1.771263,1.913237,0.831607,2.0


In [33]:
px.scatter_3d(x_train_df_pca, x='pca_1', y='pca_2', z='pca_3', color='target', template='simple_white', width=950)

In [34]:
x_test_pca[:10]

array([[-0.389 , -0.3538, -0.0821],
       [ 0.6597,  2.6431, -0.8846],
       [ 3.2379,  0.195 , -0.7813],
       [ 0.5126,  2.0658,  1.1236],
       [-2.4005, -1.0567,  1.456 ],
       [ 0.3166,  1.9054, -0.0386],
       [ 0.6566,  1.988 ,  0.5793],
       [ 3.0008, -0.2845, -0.0953],
       [-2.4109, -1.8965, -0.01  ],
       [ 2.4221, -0.131 ,  1.1714]])